<a href="https://colab.research.google.com/github/bos-hieu/ai-playground/blob/main/product_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langchain-community sentence-transformers faiss-cpu openai tiktoken

In [2]:
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
import os

# Sample furniture database (replace with your catalog)
furniture = [
    {"name": "ErgoDesk Pro", "type": "standing desk", "price": 450, "material": "bamboo", "description": "Adjustable height desk with bamboo top."},
    {"name": "ComfortChair X", "type": "ergonomic chair", "price": 350, "material": "mesh", "description": "Breathable mesh chair with lumbar support."},
    {"name": "ZenPod Mini", "type": "office pod", "price": 1200, "material": "acoustic foam", "description": "Soundproof pod for focused work."},
    {"name": "EcoDesk Lite", "type": "standing desk", "price": 300, "material": "MDF", "description": "Budget-friendly standing desk with electric height adjustment."},
    {"name": "AirLift Desk", "type": "standing desk", "price": 600, "material": "steel", "description": "Heavy-duty desk with dual motors."},
    {"name": "CloudChair Elite", "type": "ergonomic chair", "price": 500, "material": "leather", "description": "Premium leather chair with memory foam."},
    {"name": "FocusPod Pro", "type": "office pod", "price": 1500, "material": "glass", "description": "Modern glass pod with noise cancellation."}
]

# 1. Preprocess data into LangChain Documents
docs = [
    Document(
        page_content=f"Name: {f['name']}\nType: {f['type']}\nPrice: ${f['price']}\nMaterial: {f['material']}\nDescription: {f['description']}",
        metadata={"name": f["name"], "price": f["price"], "type": f["type"]}
    ) for f in furniture
]

# 2. Create embeddings and vector store
embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = FAISS.from_documents(docs, embedder)

# 3. Initialize LLM (replace with your OpenAI key or local model)
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
llm = OpenAI(temperature=0.3)  # Lower temp = more focused responses

# 4. Create recommendation chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_db.as_retriever(search_kwargs={"k": 3}),  # Retrieve top 3 results
    chain_type="stuff"
)

# 5. Test a query
query = "Find a standing desk under $500 with a bamboo top"
result = qa_chain.run(query)
print(f"Recommendation: {result}")

<ipython-input-2-f51e35802824>:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedder = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn

Recommendation:  The ErgoDesk Pro is a standing desk under $500 with a bamboo top.


In [3]:
!pip install gradio

In [4]:
import gradio as gr

def recommend_furniture(query):
    result = qa_chain.run(query)
    return result

interface = gr.Interface(
    fn=recommend_furniture,
    inputs="text",
    outputs="text",
    title="Office Furniture Recommender",
    description="Ask for recommendations on office furniture!"
)

interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f41e770a24e36a38c7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
